In [ ]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow import keras
from pennylane.templates import RandomLayers
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() # select the required dataset MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data() #select Fashion MNIST

n_epochs = 30 #number of epochs
n_layer = 1  # number of layer
n_train =100
n_test =60
n_qubits=4

PREPROCESS = True
np.random.seed(0)
tf.random.set_seed(0)

train_images = x_train[:n_train]
train_labels = y_train[:n_train]
test_images = x_test[:n_test]
test_labels = y_test[:n_test]

train_images = train_images / 255
test_images = test_images / 255

train_images = np.array(train_images[..., tf.newaxis],requires_grad=False)
test_images = np.array(test_images[..., tf.newaxis],requires_grad=False)

dev = qml.device("default.qubit", wires=4)

params = 2 * np.pi * tf.random.uniform([3,20])

@qml.qnode(dev)
def circuit(data,params,cir_no):
    cir_no=cir_no
    
    
    if cir_no==1:
        
        for j in range(n_qubits):
            qml.RY(np.pi * data[j], wires=j)
       
        for i in range(n_qubits):
                qml.RX(params[1,i], wires=i)
        for i in range(n_qubits - 1):
                qml.CNOT(wires=[i, i + 1])
            
        
        
        
    elif cir_no==2:
        for j in range(n_qubits):
            qml.RX(np.pi * data[j], wires=j)
           
        for j in range(n_qubits):
            qml.Rot(params[1,3*j],params[1,3*j+1],params[1,3*j+2], wires=j)
            for j in range(0, n_qubits):
                if j == 0:
                   for i in range(0, n_qubits - 1):
                       qml.CZ(wires=[j, i + 1])
                elif j == 1:
                     qml.CZ(wires=[j, j - 1])
                     for i in range(0, n_qubits - 2):
                         qml.CZ(wires=[j, i + 2])
                
                elif j == 2:
                     qml.CZ(wires=[j, j + 1])
                     for i in range(n_qubits - 1, 1, -1):
                         qml.CZ(wires=[j, i - 2])
                elif j == 3:
                     for i in range(n_qubits - 1, 0, -1):
                         qml.CZ(wires=[j, i - 1])
                        
                        
        
       
                        
                        
    elif cir_no==3:
         for i in range(0, n_qubits):
             qml.RX(np.pi * data[i], wires=i)
          
         for i in range(0, n_qubits):
                 qml.RX(params[1,i], wires=i)
                 qml.RZ(params[1,i], wires=i)
         qml.CRX(params[1,n_qubits], wires=[n_qubits - 1, 0])
         for i in range(n_qubits - 1, 0, -1):
                    qml.CRX(params[1,n_qubits+i], wires=[i - 1, i])
         
                
                
                
    elif cir_no==4:
             for i in range(0, n_qubits):
                qml.RY(np.pi * data[i], wires=i)
           
             
             for i in range(0, n_qubits):
                 qml.RX(params[1,i], wires=i)
                 qml.RZ(params[1,i], wires=i)

             for i in range(0,n_qubits - 1,2):
                 qml.CRX(params[1,i], wires=[i, i+1])

             for i in range(0, n_qubits):
                 qml.RX(params[1,i], wires=i)
                 qml.RZ(params[1,i], wires=i)

             for i in range(1,n_qubits - 1,2):
                 qml.CRX(params[1,i], wires=[i, i+1])
                    
             
                    
                    
    elif cir_no==5:
         for j in range(n_qubits):
             qml.RY(np.pi*data[j], wires=j)
            
         for j in range(0, n_qubits):
                 qml.U3(params[1,j], params[1,j + 1], params[1,j + 2], wires=j)
         for i in range(0, n_qubits - 1):
                 qml.CNOT(wires=[i, i + 1])
         for j in range(0, int(n_qubits / 2)):
                 qml.RY(params[1,j], wires=2 * j)
                 qml.RZ(params[1,j], wires=2 * j + 1)
         for i in range(0, n_qubits - 1):
                 qml.CNOT(wires=[i,i+1])
         for j in range(0, int(n_qubits / 2)):
                 qml.RY(params[1,j], wires=2 * j)
         for i in range(0, n_qubits - 1):
                 qml.CNOT(wires=[i, i + 1])
         for j in range(0, n_qubits):
                qml.U3(params[2,j], params[2,j + 1], params[2,j + 2], wires=j)
                
                
    elif cir_no==6:
         for j in range(n_qubits):
             qml.RY(np.pi*data[j], wires=j)
         for idx in range(n_qubits):
             qml.Hadamard(wires=idx)
         for i in range(n_qubits-1,0,-1):
             qml.CZ(wires=[i,i-1])
         for i in range(0, n_qubits):
             qml.RX(params[1,i], wires=i)
                
                
         
                 
    return [qml.expval(qml.PauliZ(w)) for w in range(4)]




def quanv(image):
    """Convolves the input image with many applications of the same quantum circuit."""
    out = np.zeros((14, 14, 4))
    for j in range(0, 28, 2):
        for k in range(0, 28, 2):
            
             q_results = circuit(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params,1
             )
             
             for i in range(4):
                out[j // 2, k // 2, i] = q_results[i]

    return out

if PREPROCESS == True:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(train_images):
        print("{}/{}        ".format(idx + 1, n_train), end="\r")
        q_train_images.append(quanv(img))
    q_train_images = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(test_images):
        print("{}/{}        ".format(idx + 1, n_test), end="\r")
        q_test_images.append(quanv(img))
    q_test_images = np.asarray(q_test_images)


model = tf.keras.models.Sequential([
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='same'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

opt=tf.keras.optimizers.Adam(learning_rate=0.01)


model.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],)


n_epochs=30


q_history = model.fit(q_train_images,train_labels,validation_data=(q_test_images, test_labels),batch_size=16,epochs=n_epochs,verbose=2)
score = model.evaluate(q_test_images, test_labels, verbose=0)


model1 = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=4, kernel_size=(2,2),input_shape=(28,28,1),activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='same'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(10, activation="softmax")
    ])
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
model1.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],)


n_epochs=30



c_history = model1.fit(
    train_images,
    train_labels,
    validation_data=(test_images, test_labels),
    batch_size=16,
    epochs=n_epochs,
    verbose=2,
)



In [ ]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from pennylane.templates import RandomLayers
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
n_epochs = 30
n_layer = 2
n_train =250 # input (50,100,250,500)
n_test =140  # input (30,60,140,280)
n_qubits=4

PREPROCESS = True
np.random.seed(0)
tf.random.set_seed(0)

train_images = x_train[:n_train]
train_labels = y_train[:n_train]
test_images = x_test[:n_test]
test_labels = y_test[:n_test]

train_images = train_images / 255
test_images = test_images / 255

train_images = np.array(train_images[..., tf.newaxis],requires_grad=False)
test_images = np.array(test_images[..., tf.newaxis],requires_grad=False)

dev = qml.device("default.qubit", wires=4)
params = 2 * np.pi * tf.random.uniform([3,20])

@qml.qnode(dev)
def circuit(data,params,cir_no):
    cir_no=cir_no
    
   
    if cir_no==1:
        
        for j in range(n_qubits):
            qml.RY(np.pi * data[j], wires=j)
       
        for i in range(n_qubits):
                qml.RX(params[1,i], wires=i)
        for i in range(n_qubits - 1):
                qml.CNOT(wires=[i, i + 1])
            
        for i in range(n_qubits):
            qml.RX(params[2,i], wires=i)
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        
        
    elif cir_no==2:
        for j in range(n_qubits):
            qml.RX(np.pi * data[j], wires=j)
           
        for j in range(n_qubits):
            qml.Rot(params[1,3*j],params[1,3*j+1],params[1,3*j+2], wires=j)
            for j in range(0, n_qubits):
                if j == 0:
                   for i in range(0, n_qubits - 1):
                       qml.CZ(wires=[j, i + 1])
                elif j == 1:
                     qml.CZ(wires=[j, j - 1])
                     for i in range(0, n_qubits - 2):
                         qml.CZ(wires=[j, i + 2])
                
                elif j == 2:
                     qml.CZ(wires=[j, j + 1])
                     for i in range(n_qubits - 1, 1, -1):
                         qml.CZ(wires=[j, i - 2])
                elif j == 3:
                     for i in range(n_qubits - 1, 0, -1):
                         qml.CZ(wires=[j, i - 1])
                        
                        
        for j in range(n_qubits):
            qml.Rot(params[2,3*j],params[2,3*j+1],params[2,3*j+2], wires=j)
            for j in range(0, n_qubits):
                if j == 0:
                   for i in range(0, n_qubits - 1):
                       qml.CZ(wires=[j, i + 1])
                elif j == 1:
                     qml.CZ(wires=[j, j - 1])
                     for i in range(0, n_qubits - 2):
                         qml.CZ(wires=[j, i + 2])
                elif j == 2:
                     qml.CZ(wires=[j, j + 1])
                     for i in range(n_qubits - 1, 1, -1):
                         qml.CZ(wires=[j, i - 2])
                elif j == 3:
                     for i in range(n_qubits - 1, 0, -1):
                         qml.CZ(wires=[j, i - 1])
       
                        
                        
    elif cir_no==3:
         for i in range(0, n_qubits):
             qml.RX(np.pi * data[i], wires=i)
          
         for i in range(0, n_qubits):
                 qml.RX(params[1,i], wires=i)
                 qml.RZ(params[1,i], wires=i)
         qml.CRX(params[1,n_qubits], wires=[n_qubits - 1, 0])
         for i in range(n_qubits - 1, 0, -1):
                    qml.CRX(params[1,n_qubits+i], wires=[i - 1, i])
                    
         for i in range(0, n_qubits):
                 qml.RX(params[2,i], wires=i)
                 qml.RZ(params[2,i], wires=i)
         qml.CRX(params[2,n_qubits], wires=[n_qubits - 1, 0])
         for i in range(n_qubits - 1, 0, -1):
                 qml.CRX(params[2,n_qubits+i], wires=[i - 1, i])
                
                
                
    elif cir_no==4:
             for i in range(0, n_qubits):
                qml.RY(np.pi * data[i], wires=i)
           
             
             for i in range(0, n_qubits):
                 qml.RX(params[1,i], wires=i)
                 qml.RZ(params[1,i], wires=i)

             for i in range(0,n_qubits - 1,2):
                 qml.CRX(params[1,i], wires=[i, i+1])

             for i in range(0, n_qubits):
                 qml.RX(params[1,i], wires=i)
                 qml.RZ(params[1,i], wires=i)

             for i in range(1,n_qubits - 1,2):
                 qml.CRX(params[1,i], wires=[i, i+1])
                    
             for i in range(0, n_qubits):
                 qml.RX(params[2,i], wires=i)
                 qml.RZ(params[2,i], wires=i)

             for i in range(0,n_qubits - 1,2):
                 qml.CRX(params[2,i], wires=[i, i+1])

             for i in range(0, n_qubits):
                 qml.RX(params[2,i], wires=i)
                 qml.RZ(params[2,i], wires=i)

             for i in range(1,n_qubits - 1,2):
                 qml.CRX(params[2,i], wires=[i, i+1])
                    
                    
    elif cir_no==5:
         for j in range(n_qubits):
             qml.RY(np.pi*data[j], wires=j)
            
         for j in range(0, n_qubits):
                 qml.U3(params[1,j], params[1,j + 1], params[1,j + 2], wires=j)
         for i in range(0, n_qubits - 1):
                 qml.CNOT(wires=[i, i + 1])
         for j in range(0, int(n_qubits / 2)):
                 qml.RY(params[1,j], wires=2 * j)
                 qml.RZ(params[1,j], wires=2 * j + 1)
         for i in range(0, n_qubits - 1):
                 qml.CNOT(wires=[i,i+1])
         for j in range(0, int(n_qubits / 2)):
                 qml.RY(params[1,j], wires=2 * j)
         for i in range(0, n_qubits - 1):
                 qml.CNOT(wires=[i, i + 1])
         for j in range(0, n_qubits):
                qml.U3(params[2,j], params[2,j + 1], params[2,j + 2], wires=j)
                
                
    elif cir_no==6:
         for j in range(n_qubits):
             qml.RY(np.pi*data[j], wires=j)
         for idx in range(n_qubits):
             qml.Hadamard(wires=idx)
         for i in range(n_qubits-1,0,-1):
             qml.CZ(wires=[i,i-1])
         for i in range(0, n_qubits):
             qml.RX(params[1,i], wires=i)
                
                
         for idx in range(n_qubits):
             qml.Hadamard(wires=idx)
         for i in range(n_qubits-1,0,-1):
             qml.CZ(wires=[i,i-1])
         for i in range(0, n_qubits):
             qml.RX(params[2,i], wires=i)
                 
        
    return [qml.expval(qml.PauliZ(w)) for w in range(4)]





def quanv(image):
    """Convolves the input image with many applications of the same quantum circuit."""
    out = np.zeros((14, 14, 4))
    for j in range(0, 28, 2):
        for k in range(0, 28, 2):
            
             q_results = circuit(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ],params,4
             )
             
             for i in range(4):
               out[j // 2, k // 2, i] = q_results[i]

    return out

if PREPROCESS == True:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(train_images):
        print("{}/{}        ".format(idx + 1, n_train), end="\r")
        q_train_images.append(quanv(img))
    q_train_images = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(test_images):
        print("{}/{}        ".format(idx + 1, n_test), end="\r")
        q_test_images.append(quanv(img))
    q_test_images = np.asarray(q_test_images)





model = tf.keras.models.Sequential([
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='same'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

opt=tf.keras.optimizers.Adam(learning_rate=0.01)


model.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],)




from tensorflow import keras
import matplotlib.pyplot as plt
n_epochs=30


q_history = model.fit(q_train_images,train_labels,validation_data=(q_test_images, test_labels),batch_size=16,epochs=n_epochs,verbose=2)
score = model.evaluate(q_test_images, test_labels, verbose=0)


model1 = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=4, kernel_size=(2,2),input_shape=(28,28,1),activation='relu'),
        tf.keras.layers.Conv2D(filters=4, kernel_size=(2,2),activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='same'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(10, activation="softmax")
    ])
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
model1.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],)
from tensorflow import keras
import matplotlib.pyplot as plt

n_epochs=30



c_history = model1.fit(
    train_images,
    train_labels,
    validation_data=(test_images, test_labels),
    batch_size=16,
    epochs=n_epochs,
    verbose=2,
)

